# KMS 資料治理系統：架構概覽
## 基於運算思維的系統設計

**課程**: CS785 - 企業級檢索增強生成系統
**模組**: 資料治理前置工作  
**學習時數**: 30 分鐘  
**作者**: Data Governance Team

---

## 🎯 學習目標

完成本教學後，你將能夠：
1. 理解為什麼 RAG 系統前需要資料治理
2. 掌握運算思維的四大原則
3. 了解 KMS 資料治理的完整架構
4. 知道如何應用到實際場景

---

## 💡 為什麼需要資料治理？

### 問題：直接用 RAG 會怎樣？

```python
# 糟糕的做法：直接餵給 RAG
documents = load_all_pdfs("./messy_folder")  # 💥 災難開始
vectorstore = Chroma.from_documents(documents)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore)

# 結果：
# - 過時的文檔被檢索 ❌
# - 低品質內容混入答案 ❌
# - 找不到相關文檔 ❌
# - 無法追蹤資料來源 ❌
```

### 解決方案：建立資料治理

```python
# 正確的做法：先治理，再 RAG
governance = DataGovernanceSystem(
    quality_threshold=0.8,
    freshness_days=90,
    security_level="internal"
)

# 1. 攝取與驗證
documents = governance.ingest_with_validation("./papers")

# 2. 品質評估
quality_passed = governance.assess_quality(documents)

# 3. 元資料豐富化
enriched_docs = governance.enrich_metadata(quality_passed)

# 4. 然後才給 RAG
vectorstore = Chroma.from_documents(enriched_docs)

# 結果：
# - 高品質文檔 ✓
# - 完整元資料 ✓
# - 可追溯來源 ✓
# - 持續監控品質 ✓
```

## 🧠 運算思維的四大原則

### 1. 分解 (Decomposition)

**將複雜問題拆解為可管理的小問題**

```
KMS 資料治理
│
├── 原始文檔收集
│   ├── 檔案格式驗證
│   ├── 病毒掃描
│   └── 初步分類
│
├── 文檔處理
│   ├── 內容提取 (Docling)
│   ├── OCR 處理
│   └── 格式標準化
│
├── 索引建立
│   ├── 文本分塊
│   ├── 向量化
│   └── 索引優化
│
├── 元資料管理
│   ├── 自動提取
│   ├── 手動標註
│   └── 標籤系統
│
├── 品質控制
│   ├── 多維度評估
│   ├── 異常檢測
│   └── 人工審核
│
└── 血緣追蹤
    ├── 處理歷史
    ├── 變更記錄
    └── 審計日誌
```

### 2. 模式識別 (Pattern Recognition)

**找出文檔處理中的共同模式**

#### 文檔類型模式

| 類型 | 命名模式 | 結構特徵 | 處理策略 |
|------|----------|----------|----------|
| 論文 | `YYYY_Title.pdf` | Abstract, References | 引用提取 + 高精度 OCR |
| 報告 | `Report_YYYY_QX.pdf` | 章節, 表格, 圖表 | 表格識別 + 圖表提取 |
| 政策 | `Policy_Code_vX.pdf` | 條款, 版本號 | 版本追蹤 + 合規檢查 |
| 手冊 | `Manual_Product.pdf` | 目錄, 步驟 | 結構化提取 |

#### 品質問題模式

```python
# 常見品質問題的模式
QUALITY_PATTERNS = {
    "too_short": lambda doc: len(doc.text) < 100,
    "scan_quality_low": lambda doc: doc.ocr_confidence < 0.7,
    "outdated": lambda doc: (datetime.now() - doc.date).days > 365,
    "missing_metadata": lambda doc: not doc.author or not doc.title,
    "encoding_error": lambda doc: "\ufffd" in doc.text,
}
```

### 3. 抽象化 (Abstraction)

**提取關鍵特徵，隱藏不必要的細節**

#### 統一的文檔抽象層

```python
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import List, Dict, Optional
from datetime import datetime

@dataclass
class Document(ABC):
    """統一的文檔抽象"""
    
    # 核心屬性
    id: str
    content: str
    metadata: Dict
    
    # 品質指標
    quality_score: float
    extraction_confidence: float
    
    # 生命週期
    status: str  # raw, processed, indexed, published
    created_at: datetime
    updated_at: datetime
    
    @abstractmethod
    def validate(self) -> bool:
        """驗證文檔是否符合標準"""
        pass
    
    @abstractmethod
    def to_chunks(self) -> List['DocumentChunk']:
        """分塊處理"""
        pass

# 具體實作
class PaperDocument(Document):
    """論文文檔"""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.metadata.update({
            "type": "paper",
            "has_abstract": True,
            "has_references": True
        })
    
    def validate(self) -> bool:
        # 論文特定的驗證邏輯
        return (
            len(self.content) > 1000 and
            "abstract" in self.content.lower() and
            self.quality_score > 0.7
        )
```

### 4. 演算法設計 (Algorithm Design)

**設計解決問題的步驟與流程**

#### 文檔處理管線演算法

```python
def document_governance_pipeline(raw_documents: List[Path]) -> List[Document]:
    """
    資料治理管線主演算法
    
    Time Complexity: O(n * m)
    - n: 文檔數量
    - m: 平均文檔處理時間
    
    Space Complexity: O(n * k)
    - k: 平均文檔大小
    """
    
    processed_docs = []
    
    for raw_doc in raw_documents:
        # Step 1: 驗證 (O(1))
        if not validate_format(raw_doc):
            log_error(f"Invalid format: {raw_doc}")
            continue
        
        # Step 2: 提取內容 (O(p) - p=頁數)
        content = extract_content(raw_doc)  # Docling
        
        # Step 3: 品質評估 (O(c) - c=字符數)
        quality = assess_quality(content)
        
        if quality.score < QUALITY_THRESHOLD:
            log_quality_issue(raw_doc, quality)
            continue
        
        # Step 4: 元資料提取 (O(c))
        metadata = extract_metadata(content, raw_doc)
        
        # Step 5: 建構文檔物件 (O(1))
        doc = create_document(
            content=content,
            metadata=metadata,
            quality=quality
        )
        
        # Step 6: 記錄血緣 (O(1))
        track_lineage(doc, raw_doc)
        
        processed_docs.append(doc)
    
    return processed_docs
```

## 📊 系統架構圖

### 資料流架構

```
┌─────────────────────────────────────────────────────────────┐
│                     01_RAW (原始層)                          │
│  papers/  reports/  policies/  manuals/                    │
└────────────────────┬────────────────────────────────────────┘
                     │
                     ▼
        ┌────────────────────────┐
        │  格式驗證 & 病毒掃描    │
        └────────────┬───────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────────┐
│               02_PROCESSED (處理層)                          │
│                                                              │
│  ┌─────────────┐  ┌──────────────┐  ┌──────────────┐      │
│  │   Docling   │  │  OCR Engine  │  │  Chunking    │      │
│  │  內容提取    │─▶│  圖像識別    │─▶│  文本分塊    │      │
│  └─────────────┘  └──────────────┘  └──────────────┘      │
└────────────────────┬────────────────────────────────────────┘
                     │
                     ▼
        ┌────────────────────────┐
        │    品質評估 (8維度)    │
        │  Threshold: 0.75      │
        └────────────┬───────────┘
                     │
          ┌──────────┴──────────┐
          │                     │
          ▼ Pass                ▼ Fail
┌─────────────────┐    ┌────────────────┐
│  元資料提取      │    │   人工審核      │
│  自動標註        │    │   品質改進      │
└────────┬────────┘    └────────────────┘
         │
         ▼
┌─────────────────────────────────────────────────────────────┐
│                 03_INDEXED (索引層)                          │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐     │
│  │ Vector Index │  │Keyword Index │  │ Hybrid Index │     │
│  │  (ChromaDB)  │  │(Elasticsearch│  │  (Combined)  │     │
│  └──────────────┘  └──────────────┘  └──────────────┘     │
└────────────────────┬────────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────────┐
│            04_METADATA (元資料層)                            │
│  document_catalog.db  │  tags/  │  schemas/                │
└─────────────────────────────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────────┐
│                  RAG 系統就緒                                 │
│  ✓ 高品質文檔  ✓ 完整元資料  ✓ 可追溯  ✓ 持續監控           │
└─────────────────────────────────────────────────────────────┘
```

## 🎓 關鍵概念總結

### 1. 資料治理 ≠ 額外負擔

資料治理是 RAG 系統成功的**前置必要工作**，不是可有可無的。

| Without Governance | With Governance |
|-------------------|------------------|
| 混亂的文檔 | 結構化的知識 |
| 不可信的結果 | 可追溯的答案 |
| 無法維護 | 持續改進 |
| 品質不穩定 | 品質保證 |

### 2. 運算思維的實際價值

- **分解**: 讓複雜系統變得可管理
- **模式識別**: 減少重複工作，自動化處理
- **抽象化**: 提高代碼復用性，降低耦合
- **演算法**: 優化性能，保證品質

### 3. 實用主義原則

> **Linus Torvalds**: "Talk is cheap. Show me the code."

- 先做簡單版本，能跑起來
- 測量實際效果，不要猜測
- 有問題再優化，不要過度設計
- 代碼要能維護，別搞複雜炫技

## 🚀 下一步

完成架構概覽後，接下來的實作模組：

1. **01_document_ingestion.ipynb** - 文檔攝取與驗證
2. **02_metadata_management.ipynb** - 元資料管理
3. **03_quality_control.ipynb** - 品質控制
4. **04_end_to_end_demo.ipynb** - 端到端 Demo

---

## 📚 參考資源

- [Ch1 企業知識治理理論](../../ch1_document_governance/lectures/01_enterprise_knowledge_governance.md)
- [文檔處理基礎](../../ch1_document_governance/lectures/01_document_processing_fundamentals.md)
- [Docling 官方文檔](https://github.com/DS4SD/docling)
- [ISO 25012 資料品質標準](https://www.iso.org/standard/35736.html)

---

**版本**: v1.0  
**更新**: 2025-01-17  
**授權**: MIT License